In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
from datetime import datetime
from zoneinfo import ZoneInfo

# 48 hours

In [ ]:
# Send a GET request to the website
url_48 = "https://weather.com/weather/hourbyhour/l/c097b546627cdff2da1e276cb9b2731055718a5e7270d777a92857a9701c7870" 
response_48 = requests.get(url_48)

# Create a BeautifulSoup object from the response content
soup_48 = BeautifulSoup(response_48.content, "html.parser")

In [ ]:
temp_val = soup_48.findAll('div', attrs={'class':'DetailsTable--field--CPpc_'})

In [ ]:
forecast = pd.DataFrame()
forecast['hour'] = np.linspace(1, 48, 48)
forecast['temp'] = [round((int(temp_val[i].text.replace('Feels Like', '').replace('°', ''))-32 ) *5/9, 1) for i in list(np.array(range(288))) if i%6 == 0]
forecast['wind_speed'] = [round(float(temp_val[i].text.split(' ')[1])*1.60934, 1) for i in list(np.array(range(288))) if i%6 == 1]
forecast['wind_direction'] = [temp_val[i].text.split(' ')[0] for i in list(np.array(range(288))) if i%6 == 1]
forecast['humidity'] = [int(temp_val[i].text.replace('Humidity', '')[:-1]) for i in list(np.array(range(288))) if i%6 == 2]
forecast['cloud_cover'] = [int(temp_val[i].text.replace('Cloud Cover', '')[:-1]) for i in list(np.array(range(288))) if i%6 == 4]
forecast['rain'] = [int(temp_val[i].text.replace('Rain Amount', '').replace(' in', '')) for i in list(np.array(range(288))) if i%6 == 5]

In [ ]:
forecast

# 10 days

In [ ]:
# Send a GET request to the website
url_10 = "https://weather.com/weather/tenday/l/c097b546627cdff2da1e276cb9b2731055718a5e7270d777a92857a9701c7870"
response_10 = requests.get(url_10)

# Create a BeautifulSoup object from the response content
soup_10 = BeautifulSoup(response_10.content, "html.parser")

In [ ]:
humidity_10days = soup_10.findAll('div', attrs={'class':'DetailsTable--field--CPpc_'})
conditions_10days = soup_10.findAll('div', attrs={'class':'DailyContent--ConditionSummary--2gdfo'})
content_10days = soup_10.findAll('div', attrs={'class':'DailyContent--DailyContent--1yRkH'})

In [ ]:
# now = datetime.now(tz=ZoneInfo("Europe/Brussels"))

tendays_df = pd.DataFrame()
tendays_df['day'] = [content_10days[i].text[:6] for i in range(len(content_10days))]
if len(tendays_df)%2 == 0:
    tendays_df['Part of day'] = ['Day', 'Night'] * 14
if len(tendays_df)%2 == 1:
    tendays_df['Part of day'] = ['Night'] + ['Day', 'Night'] * 14
tendays_df['temp'] = [round((int(conditions_10days[i].text.split('°')[0]) - 32)*5/9, 1) for i in range(len(conditions_10days))]
tendays_df['wind_speed'] = [round(int(conditions_10days[i].text.split('%')[1].split(' ')[1])*1.60934, 1) for i in range(len(conditions_10days))]
tendays_df['wind_direction'] = [conditions_10days[i].text.split('%')[1].split(' ')[0] for i in range(len(conditions_10days))]
tendays_df['humidity'] = [int(humidity_10days[i].text.replace('Humidity', '')[:-1]) for i in range(len(humidity_10days)) if i%4 == 0]

tendays_df